## Imagina se eu pudesse ter uma pontuação de cada aluno por intervalo de avaliação. Essa pontuação seria prevista por regressão ajuste de parametros ou por previsão de uma nota na prova baseado naqueles atributos que eu ja coletei

In [1]:
import re
import os
import math
import pandas as pd
import datetime

In [2]:
path = '/Volumes/MacOSExtended/Dataset/2016-1/'

In [3]:
dictHorarios = {}

In [4]:
#Coleta a data inicial e final de cada exercicio junto com o seu horario
def coletaDatas(arquivo):
    horarios = []
    hBegin = -1
    hEnd = -1
    dataIni = -1
    dataFim = -1
    horaIni = -1
    horaFim = -1
    for linha in arquivo:
        busca = re.findall(r'\d{4}-\d{1,2}-\d{1,2} \d+:\d+:\d+\.\d+',linha)
        if(len(busca) > 0):
            horaIni = busca[0]
            horarios.append(horaIni)
    if(len(horarios)>0):
        hBegin = horarios[0]
        hEnd = horarios[-1]
    if(hBegin != -1):
        busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hBegin)
        if(len(busquinha) > 0):
            dataIni = busquinha[0]
        busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hBegin)
        if(len(busquinha) > 0):
            horaIni = busquinha[0]
    if(hEnd != -1):
            busquinha = re.findall(r'\d{4}-\d{1,2}-\d{1,2}',hEnd)
            if(len(busquinha)>0):
                    dataFim = busquinha[0]
            busquinha = re.findall(r'\d+:\d+:\d+\.\d+',hEnd)
            if(len(busquinha) > 0):
                horaFim = busquinha[0]
    '''            
    print("data raw:")
    print(hBegin,hEnd)
    print(dataIni, dataFim)
    print("Datas")
    print(horaIni,horaFim)
    print("EOF")
    '''
    return horarios, dataIni, dataFim, horaIni, horaFim

In [5]:
#Calcula a quantidade de linhas de um arquivo de log
def qntd_linhas_log(arquivo):
    contador = 0
    paste = 0
    delete = 0
    for linha in arquivo.split("\n"):
        contador += 1
        busca = re.search(r'paste',linha)
        busca2 = re.search(r'delete',linha)
        busca3 = re.search(r'Backspace',linha)
        if(busca): paste += 1
        if(busca2): delete += 1
        if(busca3): delete += 1
    
    p1 = paste/contador
    p2 = delete/contador
    return contador,round(p1,2),round(p2,2)
    

In [6]:
#Calcula o tempo empenhado no exercicio
def tempoEmpenhado(data_ini, data_fim, hora_ini, hora_fim):
    s = '' + str(data_ini) + ' ' + str(hora_ini) 
    r = '' + str(data_fim) + ' ' +str(hora_fim)
    date1 = int(datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    date2 = int(datetime.datetime.strptime(r, '%Y-%m-%d %H:%M:%S.%f').strftime("%s"))
    difdate = date2 - date1
    return difdate

In [7]:
#Calcula a quantidade de dias em que o mesmo exercicio foi resolvido
def n_acessosColetor(arquivo):
    primeiroAcesso = 1
    difdate = 0
    for linha in arquivo:
        busca = re.search(r'\d{4}-\d{1,2}-\d{1,2}',linha)
        if(busca):
            if(primeiroAcesso):
                dataIni = busca.group(0)
                primeiroAcesso = 0
            dataAtual = busca.group(0)
            #date1 = int(datetime.datetime.strptime(dataIni, '%Y-%m-%d').strftime("%d"))
            #date2 = int(datetime.datetime.strptime(dataAtual, '%Y-%m-%d').strftime("%d"))
            #difdate = date2 - date1
            start_date = datetime.datetime.strptime(dataIni, '%Y-%m-%d')
            end_date = datetime.datetime.strptime(dataAtual, '%Y-%m-%d')
            difdate = abs(end_date-start_date)
            difdate = difdate.days
            

            
    
    return difdate

In [8]:
#Ve a quantidade de submissoes em um exercicio
def attemptsColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'SUBMITION',linha)
        if(busca): contador += 1
    return contador

In [9]:
#Ve a quantidade de testes em um exercicio
def testColetor(arquivo):
    contador = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'TEST',linha)
        if(busca): contador += 1
    return contador

In [10]:
#Ve a quantidade de exercicios em um laboratorio
def exercicios_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'number of exercises: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [11]:
#Ve a quantidade de exercicios respondidos corretamente em um laboratorio
def exer_certos_no_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'correct: \d+',linha)
        if(busca): 
            busca2 = re.search(r'\d+',busca.group(0))
            return busca2.group(0)

In [12]:
#Nota final por laboratorio
def nota_lab(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'grade \(0-10\): \d+\.\d+',linha)
        if(busca):
            busca2 = re.search(r'\d+\.\d+',busca.group(0))
            return busca2.group(0)

In [13]:
#Quantidade de linhas de um codigo
def qntd_linhas(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\w*',linha)
        if(busca): qntd += 1
    return qntd

In [14]:
#Quantidade de linhas de um codigo sem comentarios etc
def qntd_linhas_filtrado(arquivo):
    qntd = 0
    for linha in arquivo.split("\n"):
        busca = re.search(r'\S',linha)
        if(busca): 
            busca2 = re.search(r'^(?!#).*',busca.group(0))
            if(busca2): qntd += 1

    return qntd

In [15]:
def descobre_atividade(arquivo,exercicio):
    for linha in arquivo.split("\n"):
        busca = re.search(r'type: exam',linha)
        if(busca): 
            busca2 = re.search(r'\d+',exercicio)
            return busca2.group(0)

In [16]:
def coleta_nota_grade(arquivo):
     for linha in arquivo.split("\n"):
        nota = re.search(r'\: \d+.*\d*',linha)
        if(nota):
            notaLimpa = re.search(r'\d+.*\d*',nota.group(0))
            return notaLimpa.group(0)

In [17]:
def coleta_corretos_grade(arquivo):
    for linha in arquivo.split("\n"):
        corretos = re.search(r'\bcorrect\b: \d+',linha)
        if(corretos):
            corretos = re.search(r'\d+',corretos.group(0))
            return corretos.group(0)

In [18]:
def coleta_incorretos_grade(arquivo):
    for linha in arquivo.split("\n"):
        incorretos = re.search(r'\bincorrect\b: \d+',linha)
        if(incorretos):
            incorretos = re.search(r'\d+',incorretos.group(0))
            return incorretos.group(0)

In [19]:
def coleta_brancos_grade(arquivo):
    for linha in arquivo.split("\n"):
        brancos = re.search(r'blank: \d+',linha)
        if(brancos):
            brancos = re.search(r'\d+',brancos.group(0))
            return brancos.group(0)

In [20]:
def coletor_grade(arquivo):
    for linha in arquivo.split("\n"):
        nota = re.search(r'\: \d+.\d+',linha)
        corretos = re.search(r'\bcorrect\b: \d+',linha)
        incorretos = re.search(r'\bincorrect\b: \d+',linha)
        brancos = re.search(r'blank: \d+',linha)
        if(nota):
            notaLimpa = re.search(r'\d+.\d+',nota.group(0))
            notaF = notaLimpa.group(0)
        if(corretos):
            corretos = re.search(r'\d+',corretos.group(0))
            corretosF = corretos.group(0)
        if(incorretos):
            incorretos = re.search(r'\d+',incorretos.group(0))
            incorretosF = incorretos.group(0)
        if(brancos):
            brancos = re.search(r'\d+',brancos.group(0))
            brancosF = brancos.group(0)
    print(notaF,corretosF,incorretosF,brancosF)
   
        

In [21]:
#Definindo o DataFrame
colunas = ['Aluno','Lab','Exercicio','Duracao','Qntd_Linhas','Qntd_Linhas_Filtrado','Testes','Submissoes','N_Dias','PP_Paste','PP_Delete','Linhas_Log','Nota']
df = pd.DataFrame(columns = colunas)
print(df)
df = df[0:0]

Empty DataFrame
Columns: [Aluno, Lab, Exercicio, Duracao, Qntd_Linhas, Qntd_Linhas_Filtrado, Testes, Submissoes, N_Dias, PP_Paste, PP_Delete, Linhas_Log, Nota]
Index: []


In [22]:
colunas = ['Aluno','Lab','Corretos','Incorretos','Branco','Nota']
dfGrade = pd.DataFrame(columns= colunas)

In [23]:
for folder in os.listdir(path):
    if not folder.startswith('.'):
        
        directory = os.path.join(path,folder)+'/'
        turma = folder
        PastaAtividades = directory + 'assessments'
        pastaAlunos = directory + 'users'
        for file in os.listdir(PastaAtividades):
            vetorProvas = []
            numProva = file
            file = os.path.join(PastaAtividades,file)
            arquivo = open(file,'r',encoding='ISO-8859-1')
            data = arquivo.read()
            prova = descobre_atividade(data,numProva)
            if(prova != None): vetorProvas.append(prova)
        
        for alunoUnidade in os.listdir(pastaAlunos):
            aluno = alunoUnidade
            pastaAluno = pastaAlunos + '/' + aluno
            pastaNotas = pastaAluno + '/grades'
            
            for prova in os.listdir(pastaNotas):
                file = os.path.join(pastaNotas,prova)
                try:
                    arquivo = open(file,'r',encoding='ISO-8859-1')
                    data = arquivo.read()
                    brancos = coleta_brancos_grade(data)
                    corretos = coleta_corretos_grade(data)
                    incorretos = coleta_incorretos_grade(data)
                    nota = coleta_nota_grade(data)
                    if(nota != None):
                        labson = re.search(r'\d+',prova)
                        labson = labson.group(0)
                        dfGrade = dfGrade.append({'Aluno': aluno,'Lab' : labson,'Corretos':corretos,'Incorretos':incorretos,'Branco': brancos,'Nota':nota}, ignore_index=True)
                except FileNotFoundError:
                    print(file," nao existe")
        

In [24]:
dfGrade['Aluno'] = dfGrade['Aluno'].astype(int)
dfGrade['Lab'] = dfGrade['Lab'].astype(int)
dfGrade['Corretos'] = dfGrade['Corretos'].astype(int)
dfGrade['Incorretos'] = dfGrade['Incorretos'].astype(int)
dfGrade['Branco'] = dfGrade['Branco'].astype(int)
dfGrade['Nota'] = dfGrade['Nota'].astype(float) 

In [25]:
df_csv = pd.read_csv('dadosLab2016.csv')

In [26]:
df_csv = df_csv.drop(['Nota'],axis = 1)

In [27]:
df_csv.columns

Index(['Aluno', 'Lab', 'Duracao', 'Qntd_Linhas', 'Qntd_Linhas_Filtrado',
       'Testes', 'Submissoes', 'PP_Paste', 'PP_Delete', 'Linhas_Log'],
      dtype='object')

In [28]:
dfGrade.columns

Index(['Aluno', 'Lab', 'Corretos', 'Incorretos', 'Branco', 'Nota'], dtype='object')

In [29]:
df

,Aluno,Lab,Exercicio,Duracao,Qntd_Linhas,Qntd_Linhas_Filtrado,Testes,Submissoes,N_Dias,PP_Paste,PP_Delete,Linhas_Log,Nota


In [30]:
df_unido = (pd.merge(df_csv,dfGrade, on=['Aluno','Lab']))

In [31]:
df_unido

,Aluno,Lab,Duracao,Qntd_Linhas,Qntd_Linhas_Filtrado,Testes,Submissoes,PP_Paste,PP_Delete,Linhas_Log,Corretos,Incorretos,Branco,Nota
0,1058,49,762.000000,7.000000,5.200000,1.400000,1.400000,0.008000,0.140000,275.400000,4,0,6,4.00
1,1231,27,39286.714286,5.857143,4.000000,2.285714,2.285714,0.004286,0.208571,453.000000,7,0,0,10.00
2,1231,42,103400.700000,12.000000,6.300000,3.100000,4.100000,0.002000,0.205000,694.900000,10,0,0,10.00
3,1231,48,1398.500000,10.000000,6.000000,1.000000,5.000000,0.000000,0.310000,805.500000,2,0,0,10.00
4,1231,74,612.571429,10.142857,8.000000,2.285714,2.000000,0.000000,0.177143,557.142857,7,0,0,10.00
5,1231,94,215973.500000,16.000000,11.000000,63.000000,16.500000,0.005000,0.230000,770.000000,2,0,0,10.00
6,1231,103,51502.222222,26.666667,24.666667,38.888889,13.777778,0.005556,0.242222,1448.444444,9,0,0,10.00
7,1231,122,441.500000,17.500000,16.500000,6.000000,2.000000,0.010000,0.245000,609.000000,2,0,0,10.00
8,1231,131,67557.363636,13.272727,10.909091,36.000000,12.727273,0.002727,0.188182,865.727273,11,0,0,10.00
9,1231,148,2367.500000,13.000000,11.500000,41.500000,9.000000,0.000000,0.215000,1153.000000,2,0,0,10.00


In [32]:
df_unido[df_unido['Nota']>5.0]

,Aluno,Lab,Duracao,Qntd_Linhas,Qntd_Linhas_Filtrado,Testes,Submissoes,PP_Paste,PP_Delete,Linhas_Log,Corretos,Incorretos,Branco,Nota
1,1231,27,39286.714286,5.857143,4.000000,2.285714,2.285714,0.004286,0.208571,453.000000,7,0,0,10.00000
2,1231,42,103400.700000,12.000000,6.300000,3.100000,4.100000,0.002000,0.205000,694.900000,10,0,0,10.00000
3,1231,48,1398.500000,10.000000,6.000000,1.000000,5.000000,0.000000,0.310000,805.500000,2,0,0,10.00000
4,1231,74,612.571429,10.142857,8.000000,2.285714,2.000000,0.000000,0.177143,557.142857,7,0,0,10.00000
5,1231,94,215973.500000,16.000000,11.000000,63.000000,16.500000,0.005000,0.230000,770.000000,2,0,0,10.00000
6,1231,103,51502.222222,26.666667,24.666667,38.888889,13.777778,0.005556,0.242222,1448.444444,9,0,0,10.00000
7,1231,122,441.500000,17.500000,16.500000,6.000000,2.000000,0.010000,0.245000,609.000000,2,0,0,10.00000
8,1231,131,67557.363636,13.272727,10.909091,36.000000,12.727273,0.002727,0.188182,865.727273,11,0,0,10.00000
9,1231,148,2367.500000,13.000000,11.500000,41.500000,9.000000,0.000000,0.215000,1153.000000,2,0,0,10.00000
10,1231,163,64932.454545,14.909091,12.818182,12.090909,8.090909,0.001818,0.215455,746.363636,11,0,0,10.00000


In [33]:
df_unido.to_csv('dadosProvas2016.csv', encoding='utf-8', index=False)